In [39]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
import matplotlib.ticker as ticker 
import matplotlib.pyplot as plt
from pathlib import Path
import os
import isodate
import psycopg2
from sqlalchemy import create_engine

## Data collection and storage

In [3]:
api_key = Path("YT_API_KEY.txt").read_text() # store YouTube API key in YT_API_KEY.txt
channel_handle = '@alextheanalyst' # YouTube Channel 'Alex The Analyst'
                   
api_service_name = 'youtube'
api_version = 'v3'

# Initialize YouTube Data API client using google api library
youtube = build(api_service_name, api_version, developerKey=api_key)

### Function to request channel information

In [4]:
# request channel information using youtube data api
def request_channel_info(youtube, channel_handle):    
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        forHandle=channel_handle        
        )
    
    response = request.execute()
    #return response   
        
    response_data = response['items'][0]
    data = {'channel': response_data['snippet']['title'],
            'description': response_data['snippet']['description'],
            'subscribers': response_data['statistics']['subscriberCount'],
            'views': response_data['statistics']['viewCount'],
            'total_videos': response_data['statistics']['videoCount'],
            'playlist_id': response_data['contentDetails']['relatedPlaylists']['uploads']            
            }              
        
        
    #return the statistic data of Channel 'Alex The Analyst'
    return data

In [5]:
channel_statistics = request_channel_info(youtube,channel_handle)
# print(channel_statistics)

In [6]:
# convert channel data to panda DataFrame
channel_data = pd.DataFrame([channel_statistics])
#channel_data

In [7]:
# Save channel data to a csv file
os.makedirs("data",exist_ok=True)
channel_data.to_csv('data/channel.csv', index=False)

### Funtion to get video IDs

In [8]:
def get_playlist_videos(youtube, play_list_id): 
    
    video_ids = []
    next_page_token = None   
    
    while True: 
        request = youtube.playlistItems().list(
            part='snippet,contentDetails',
            maxResults=50,
            pageToken=next_page_token,
            playlistId=play_list_id
            )
        
        response = request.execute()
        #return response
    
        for item in response['items']:
            video_id = item['contentDetails']['videoId']               
            video_ids.append(video_id)
        
        next_page_token=response.get('nextPageToken')
        #print(next_page_token)
        if not next_page_token:
            break    
           
    return video_ids 
   

In [9]:
playlist_ID = channel_data.loc[channel_data['channel']=='Alex The Analyst','playlist_id'].iloc[0]

In [10]:
video_ids = get_playlist_videos(youtube, playlist_ID)
#print(video_ids)

### Function to get video details

In [12]:
def get_video_block_details(youtube, video_block_ids):
    """
    get video details with maximal 50 video IDs
    """
    
    request = youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=','.join(video_block_ids)
        )
    response = request.execute()
    
    return response


In [13]:
def get_all_video_details(youtube,video_ids):
    """
    get details of all videos
    """
    
    video_details = []
    video_len=len(video_ids)
    
    for i in range(0,video_len,50):
        video_block_ids=video_ids[i:i+50]        
        video_details.append(get_video_block_details(youtube, video_block_ids))   

    return video_details   
        

In [14]:
video_details = get_all_video_details(youtube,video_ids)
# print(video_details)

### Function to get video statistics

In [15]:
def get_video_stats(youtube,video_ids):
    """
    get the statistics of provided videos
    """
    
    all_video_stats = []
    video_details = get_all_video_details(youtube,video_ids)
    
    for details in video_details:
        for item in details['items']:                     
            stats_data = {'video_id': item['id'],
                          'title': item['snippet']['title'],
                          'published_at':item['snippet']['publishedAt'],                          
                          'views': item['statistics'].get('viewCount',0),
                          'likes': item['statistics'].get('likeCount',0),                      
                          'comments': item['statistics'].get('commentCount',0),  
                          'tags': item['snippet'].get('tags', []),
                          'duration': item['contentDetails']['duration']        
                         }
            all_video_stats.append(stats_data)
        
    return all_video_stats
        

In [16]:
all_video_stats = get_video_stats(youtube,video_ids)
# print(all_video_stats)

In [17]:
video_data = pd.DataFrame(all_video_stats)
# print(video_data)

In [20]:
# convert duration to seconds and only keep column duration_seconds
video_data['duration_seconds'] = video_data['duration'].apply(lambda x: int(isodate.parse_duration(x).total_seconds()))
video_data = video_data.drop(columns='duration')

In [21]:
# save video data into video.csv file
video_data.to_csv('data/video.csv', index=False)

### Load data into PostgreSQL Database youtube_analysis

In [43]:

# load database credentials from a credential file

def load_db_credentials(filepath):
    credentials = {}
    with open(filepath,'r') as file:
        lines = file.readlines()
        for line in lines:
            key, value = line.strip().split('=',1)
            credentials[key.strip()] = value.strip()
            
    return credentials

# Use credentials to test connection
creds = load_db_credentials('db_credentials.txt')    

try:
    conn = psycopg2.connect(**creds)
    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    print('Connection successful.')

except Exception as e:
    print('Connection failed:', e)
    
finally:
    if conn:
        conn.close() 
        
        

# Build connection string using credentials
conn_str = f"postgresql://{creds['user']}:{creds['password']}@{creds['host']}:{creds['port']}/{creds['dbname']}"

# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Load CSV to DataFrame
df = pd.read_csv("data/video.csv")

# Write DataFrame to PostgreSQL
df.to_sql("videos", engine, if_exists="append", index=False)

        
      

Connection successful.


365